In [1]:
!pip install scrapy
!pip install response
!pip install pandas
import time
import re
import response
import requests
import pandas as pd
import numpy as np
from scrapy.http import TextResponse

     |████████████████████████████████| 245kB 2.8MB/s 
     |████████████████████████████████| 235kB 8.8MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 3.1MB 12.9MB/s 
     |████████████████████████████████| 3.2MB 31.4MB/s 
     |████████████████████████████████| 2.7MB 34.1MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=a63f2e42fb9119a363f2fdb3ce8fc22dba9b00752cce06fc4164187fa6883e5a
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=adcf25050786c3d1afe5986605d38bafb5c68fcec8508a9a787f6aa20e50ea43
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatcher protego


In [2]:
url = "https://staff.am/en/jobs"
base_url = "https://staff.am"

In [3]:
class Announcments:
    def __init__(self,url):
        self.url = url
        self.page = requests.get(self.url)
        self.response = TextResponse(body=self.page.text,url=self.url,encoding="utf-8")

    def get_vacancy(self):
        vac = self.response.xpath('//div[@class="job-inner job-item-title"]/p[@class="font_bold"]/text()').extract()
        return vac 

    def get_company(self):
        comp = self.response.xpath('//div[@class="job-inner job-item-title"]/p[@class="job_list_company_title"]/text()').extract()
        return comp

    def get_deadline(self):
        dl1 = self.response.css('div[class="job-inner job-list-deadline"] p::text').extract()
        dl2 = [''.join(x) for x in zip(dl1[0::2], dl1[1::2])]
        del dl2[1::2]
        dl = [i.replace("\n\n", "").replace("\n"," ").strip() for i in dl2]
        return dl 

    def get_location(self):
        loc = self.response.xpath('//div[@class="job-inner job-list-deadline"]/p[@class="job_location"]/text()').extract()
        loc = [i.replace('\n','').strip() for i in loc]
        return loc 

    def get_ind_page(self):
        ind_page = [base_url + i for i in self.response.xpath('//div[@class="list-view"]/div/div/a/@href').extract()]
        return ind_page

    def get_next(self):
        page = self.response.xpath('//ul[@class="pagination"]/li[@class="next"]/a/@href').extract()
        return page

In [4]:
j = Announcments(url)
vacancies = []
companies = []
deadlines = []
locations = []
i_pages = []
announcments = []

In [5]:
while True:
    if(j.get_next() == []):
        vacancies = vacancies + j.get_vacancy()
        companies = companies + j.get_company()
        deadlines = deadlines + j.get_deadline()
        locations = locations + j.get_location()
        i_pages = i_pages + j.get_ind_page()
        break
    else:
        vacancies = vacancies + j.get_vacancy()
        companies = companies + j.get_company()
        deadlines = deadlines + j.get_deadline()
        locations = locations + j.get_location()
        i_pages = i_pages + j.get_ind_page()
        url = base_url + j.get_next()[0]
        j = Announcments(url)

In [6]:
locations = [i for i in locations if len(i)>0]


In [7]:
announcments.append(vacancies)
announcments.append(companies)
announcments.append(deadlines)
announcments.append(locations)
announcments.append(i_pages)

In [8]:
announcments = list(map(list, zip(*announcments)))
announcments_df = pd.DataFrame(announcments, columns=['Vacancies','Companies','Deadlines','Locations','Individual Pages'])

In [11]:
from google.colab import files
announcments_df.to_csv('HW3_2.csv')
files.download('HW3_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
a = announcments_df['Companies'].value_counts().idxmax()
a_company = []
for i in announcments_df['Companies']:
    if(i == a):
        a_company.append(i)

In [13]:
print(a)


Digitain 


In [14]:
str(len(a_company))


'33'

In [15]:
ad = []
for i in announcments_df['Vacancies']:
    if("Data" in i or "DATA" in i):
        ad.append(i)

In [16]:
print(ad)

['Data Scientist', 'Data Engineer (Spark)', 'Data analyst', 'Data Engineer', 'Junior Database Administrator', 'Senior Database Administrator', '(SENIOR) DATA/DATABASE ARCHITECT (M/F/D)', 'Data Specialist / QA Engineer', 'Database Administrator', 'Engineering Manager, Java Microservices & Data']


In [17]:
str(len(ad))


'10'

10 job announcments have the word "Data" inside.